In [1]:
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from transformers import AutoTokenizer
import random
from prompt_templates import create_prompt
from transformers import MT5ForConditionalGeneration
import argparse
from datetime import datetime
from models.get_model import get_model
from dataloader import get_loader
import torch
from torch import nn
from evaluate import evaluate
import copy
import wandb
from tqdm import tqdm
import os
import json
from torch.nn import CrossEntropyLoss
from transformers import AutoTokenizer
from scheduler import CosineWarmupScheduler
from utils import compute_rank_loss
from torch.cuda.amp import GradScaler


ImportError: cannot import name 'create_prompt' from 'prompt_templates' (/teamspace/studios/this_studio/clean/RecSys/code/prompt_templates.py)

In [6]:
%load_ext autoreload
%autoreload 2

In [2]:
from transformers import MT5ForQuestionAnswering
from transformers import AutoTokenizer


In [3]:
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")
model = MT5ForQuestionAnswering.from_pretrained("google/mt5-small")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Some weights 

In [12]:
i = tokenizer("hallo dit is de input hahahah")
o1 = tokenizer("dit is > @ bier @ ll @ deze @ hallo")
o2 = tokenizer("dit is @ bier @ ll @ deze @ hallo")

In [8]:
import torch

In [5]:
device = "cuda"

In [6]:
model.to(device)

MT5ForQuestionAnswering(
  (shared): Embedding(250112, 512)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 512)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              

In [13]:
model.eval()
output1 = model(input_ids=torch.tensor([i["input_ids"]]).to(device), 
    attention_mask=torch.tensor([i["attention_mask"]]).to(device),
    decoder_input_ids=torch.tensor([o1["input_ids"]]).to(device)
)
output2 = model(input_ids=torch.tensor([i["input_ids"]]).to(device), 
    attention_mask=torch.tensor([i["attention_mask"]]).to(device),
    decoder_input_ids=torch.tensor([o2["input_ids"]]).to(device)
)

In [14]:
output1.start_logits, output2.start_logits

(tensor([[0.6349, 0.5556, 0.2165, 0.7735, 0.5941, 0.7483, 0.4126, 0.2228, 0.0319,
          0.0325, 0.1985, 0.0472, 0.1500, 1.7687, 0.6690]], device='cuda:0',
        grad_fn=<CloneBackward0>),
 tensor([[ 0.6349,  0.5556,  0.2811, -0.0169, -0.0850,  0.1010,  0.1000,  0.0316,
           0.0485,  0.1231, -0.4491,  0.3784,  1.9469,  0.6765]],
        device='cuda:0', grad_fn=<CloneBackward0>))

In [11]:
o1["input_ids"], o2["input_ids"] 

([1568,
  339,
  1250,
  259,
  69214,
  1250,
  259,
  1578,
  1250,
  3675,
  1250,
  18691,
  268,
  1],
 [1568,
  339,
  1250,
  259,
  69214,
  1250,
  259,
  1578,
  1250,
  3675,
  1250,
  18691,
  268,
  1])

In [8]:
tokenizer("> @")

{'input_ids': [691, 1250, 1], 'attention_mask': [1, 1, 1]}

In [9]:
import torch

In [15]:
torch.nn.functional.one_hot(torch.tensor([1]), num_classes=10)

tensor([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]])

In [16]:
import numpy as np

In [45]:
k = 5
def ndcg_at_k(scores, labels):
    best = dcg_score_at_k(labels, labels)
    actual = dcg_score_at_k(scores, labels)
    assert actual <= best, f"{actual} > {best}"
    return actual / best

def dcg_score_at_k(scores, labels, k=5):
    k = min(len(labels), k)
    order = np.argsort(scores)[::-1]
    y_true = np.take(labels, order[:k])
    gains = 2**y_true - 1
    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gains / discounts)

def mean_squared_error_at_k(y_true, y_pred, k=5):
    order = np.argsort(y_pred)[::-1][:k]
    return mean_squared_error(np.take(y_true, order), np.take(y_pred, order))

In [44]:
from sklearn.metrics import mean_squared_error

In [51]:
labels = np.array([0,1,0,0,0])
scores = np.array([0,1.1,0,0,0])
ndcg_at_k(scores, labels), mean_squared_error_at_k(scores, labels)

(1.0, 0.0020000000000000035)

In [1]:
import torch

In [2]:
a = torch.randn((6, 5))

In [3]:
a

tensor([[-0.1474, -1.7000, -0.0734,  1.6318, -0.3340],
        [ 1.9534,  1.5190, -0.0550, -0.4541,  1.3941],
        [ 1.9743, -1.0141,  1.6381, -0.1412,  0.9967],
        [ 0.2267,  0.1350,  0.1748, -0.0435,  0.4368],
        [ 1.1042, -1.5560, -1.3872, -0.0162, -0.5325],
        [-0.2042,  0.6496, -1.6553, -0.1083, -0.6658]])

In [4]:
b = torch.tensor([3,2,1,1,1,2])
a[torch.arange(a.size(0)), b]

tensor([ 1.6318, -0.0550, -1.0141,  0.1350, -1.5560, -1.6553])

In [44]:
class b:
    dataset = "demo"
    batch_size = 3
    num_workers = 4

b = b()

In [18]:
data_loader = get_loader(b, 'train', tokenizer, T=4, debug=True)

In [45]:
data_loader2 = get_loader(b, 'train', tokenizer, T=4, debug=False)


In [46]:
out1= next(iter(data_loader2))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Asking to truncate to max_le

In [64]:
out1['pos_input_ids'].shape, out1['pos_attention_mask'].shape, out1['neg_input_ids'].shape, out1['pos_labels'].shape, out1['neg_attention_mask'].shape

(torch.Size([3, 230]),
 torch.Size([3, 230]),
 torch.Size([3, 223]),
 torch.Size([3, 2]),
 torch.Size([3, 223]))

In [51]:
for i in range(3):
    print(tokenizer.decode(out1['pos_input_ids'][i].tolist()))
    print(tokenizer.decode(out1['neg_input_ids'][i].tolist()))  

En bruger har for nylig læst artikler: Tomas Axnér bliver ny cheftræner i Team Esbjerg fra sommeren 2024, hvor han afløser Jesper Jensen. Svenskeren skal sideløbende være landstræner, Rafael Nadal har indkaldt til pressemøde torsdag. Ifølge AS og Marca vil han annoncere French Open-afbud, James Milner, Roberto Firmino, Naby Keita og Alex Oxlade-Chamberlain stopper i Liverpool efter denne sæson, Rafael Nadal har indkaldt til pressemøde torsdag. Ifølge AS og Marca vil han annoncere French Open-afbud, vil brugeren læse artiklen God fangst på havn trods muligt forsøg på at aflede? (ja/nej)</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
En bruger har for nylig læst artikler: Tomas Axnér bliver ny cheftræner i Team Esbjerg fra sommeren 2024, hvor han afl

In [65]:
pos_outputs = model(
    input_ids=out1["pos_input_ids"], 
    attention_mask=out1["pos_attention_mask"],
)
pos_outputs.encoder_last_hidden_state.shape

ValueError: You have to specify either decoder_input_ids or decoder_inputs_embeds

In [67]:
tokenizer.encode("ja / nej"), tokenizer.encode("ja"), tokenizer.encode("nej")

([432, 259, 275, 3810, 1], [432, 1], [3810, 1])

In [62]:
neg_outputs = model(
    input_ids=out1["neg_input_ids"], 
    attention_mask=out1["neg_attention_mask"],
    labels=out1["neg_labels"],
)
neg_outputs.encoder_last_hidden_state.shape

torch.Size([3, 223, 512])

In [54]:
pos_outputs.logits.shape

torch.Size([3, 2, 250112])

In [30]:
out1.keys()

dict_keys(['pos_input_ids', 'pos_attention_mask', 'neg_input_ids', 'neg_attention_mask', 'pos_labels', 'neg_labels', 'empt'])

In [43]:
print(tokenizer.decode(out1['pos_input_ids'][0].tolist()))
print(tokenizer.decode(out1['neg_input_ids'][0].tolist()))

En bruger har for nylig læst artikler: Tennis-etteren Novak Djokovic havde flotte ord om Holger Rune på pressemødet efter de to tennisstjerners dyst i Rom og fortæller, at det er ham, der skal have råd fra Rune og ikke omvendt, Rafael Nadal har indkaldt til pressemøde torsdag. Ifølge AS og Marca vil han annoncere French Open-afbud, Rygtemøllen arbejder på højtryk, efter Belarus' præsident, Alexander Lukasjenko, ikke er blevet set offentligt i en uge, Luksuslivet er slut. Bruger penge på Easyjet, sandkage og cola, vil brugeren læse artiklen - Hvis nogen tror, at vi lægger os fladt ned mod FC Nordsjælland, så må det være personer, der ikke er involveret i elitesport, siger en vred Carsten V. Jensen? (ja/nej)</s>
En bruger har for nylig læst artikler: Tennis-etteren Novak Djokovic havde flotte ord om Holger Rune på pressemødet efter de to tennisstjerners dyst i Rom og fortæller, at det er ham, der skal have råd fra Rune og ikke omvendt, Rafael Nadal har indkaldt til pressemøde torsdag. If

"En bruger har for nylig læst artikler: Tennis-etteren Novak Djokovic havde flotte ord om Holger Rune på pressemødet efter de to tennisstjerners dyst i Rom og fortæller, at det er ham, der skal have råd fra Rune og ikke omvendt, Rafael Nadal har indkaldt til pressemøde torsdag. Ifølge AS og Marca vil han annoncere French Open-afbud, Rygtemøllen arbejder på højtryk, efter Belarus' præsident, Alexander Lukasjenko, ikke er blevet set offentligt i en uge, Luksuslivet er slut. Bruger penge på Easyjet, sandkage og cola, vil brugeren læse artiklen Aktien er faldet med knap 30 procent onsdag? (ja/nej)</s>"

In [37]:
tokenizer.encode("hello")

[124393, 1]

In [27]:
out1, out2 = next(iter(data_loader2)), next(next(iter(data_loader2)))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/home/wouter/miniconda3/envs/RecSys/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/home/wouter/miniconda3/envs/RecSys/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is dep

TypeError: 'dict' object is not an iterator